Ten notatnik służył do analizy danych - jest to dokumentacja dlaczego i w jaki sposób skonstruowano finalne dane, które przetwarza model. Większą jego część zastąpił odpowiedni pipeline w modelu, który zapisuje potrzebne dane tak, aby podczas inferencji można było łatwo dopasować nowe dane do potrzebnego formatu.

# Importy

In [36]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import re
from collections import Counter
import ast
from sklearn.model_selection import train_test_split
import joblib
from math import radians, sin, cos, sqrt, atan2

In [ ]:
TRAIN_DATA_PATH = "../data/train_data.csv"
TEST_DATA_PATH = "../data/test_data.csv"

# Podstawowe wczytanie

In [ ]:
# podstawowe wczytanie plików z dostarczonego excela
listings = pd.read_csv('../data/raw/listings.csv', sep=';')
calendar = pd.read_csv('../data/raw/calendar.csv', sep=';')

# usunięcie $ i konwersja price na float
calendar['price'] = calendar['price'].str.replace(r'[\$,]', '', regex=True).astype(float)

In [39]:
# zostawiamy tylko wybrane kolumny
columns_to_keep = [
    'id',
    'neighbourhood_cleansed',
    'neighbourhood_group_cleansed',
    'latitude',
    'longitude',
    'property_type',
    'room_type',
    'accommodates',
    'bathrooms',
    'bathrooms_text',
    'bedrooms',
    'beds',
    'amenities',
    'instant_bookable',
    'price'
]

listings = listings[columns_to_keep]

In [40]:
print(listings.head())

            id           neighbourhood_cleansed neighbourhood_group_cleansed  \
0     16989407                la Vila de Gràcia                       Gràcia   
1     34133454                la Vila de Gràcia                       Gràcia   
2  1,04185E+18           la Dreta de l'Eixample                     Eixample   
3  6,95612E+17                      el Poblenou                   Sant Martí   
4       847195  l'Antiga Esquerra de l'Eixample                     Eixample   

    latitude  longitude                property_type        room_type  \
0  41.409920   2.157330         Private room in home     Private room   
1  41.397630   2.159340    Entire serviced apartment  Entire home/apt   
2  41.394798   2.165613           Entire rental unit  Entire home/apt   
3  41.399490   2.202610  Private room in rental unit     Private room   
4  41.385080   2.155270           Entire rental unit  Entire home/apt   

   accommodates  bathrooms    bathrooms_text  bedrooms  beds  \
0             2 

# Dodanie ceny (target)

In [41]:
# chcemy dla każdego listingu mieć jedną cenę (price z listings_excel_ready nie spełnia tego, bo a)to jest 'local currency', a b)są braki dla niektórych listingów
# można usunąć tą kolumnę
listings.drop(columns=['price'], inplace=True)
# więc sprawdzamy czy każdy listing z listings_excel_ready ma przynajmniej jeden wpis w calendar
listings_ids = set(listings['id'])
calendar_listing_ids = set(calendar['listing_id'])

missing_in_calendar = listings_ids - calendar_listing_ids

if missing_in_calendar:
    print(f"Brak wpisów w calendar dla listingów o ID: {missing_in_calendar}")
else:
    print("Każdy listing z listings_excel_ready ma wpis w calendar.")

Każdy listing z listings_excel_ready ma wpis w calendar.


In [42]:
# skoro tak to możemy tego użyć - chcemy połączyć listingi z odpowiednią ceną, sprawdzamy ile jest unikalnych wartości
price_stats = calendar.groupby('listing_id')['price'].agg(
    nunique='nunique',
    min='min',
    max='max',
    mean='mean',
    median='median'
).reset_index()

# niektóre listingi mają różne ceny w zależnosci od dnia - my chcemy jedną wartość, dlatego dla tych przypadków bierzemy medianę
price_stats['final_price'] = price_stats.apply(
    lambda row: row['median'] if row['nunique'] > 1 else row['min'],
    axis=1
)

In [43]:
# dodajemy kolumnę
listings = listings.merge(price_stats[['listing_id', 'final_price']], left_on='id', right_on='listing_id', how='left')

listings.drop(columns=['listing_id'], inplace=True)
listings.rename(columns={'final_price': 'price'}, inplace=True)

# Zapisanie danych

In [ ]:
# Na tym etapie można zapisać dane testowe
train_df, test_df = train_test_split(listings, test_size=0.1, random_state=42)

train_df.to_csv(TRAIN_DATA_PATH, index=False, sep=';')
test_df.to_csv(TEST_DATA_PATH, index=False, sep=';')

# Uzupełnienie brakujących danych

In [45]:
# w tych kolumnach dane są niepełne
columns_with_nan = listings.columns[listings.isnull().any()].tolist()
print('Kolumny, w których są puste wartości: ', columns_with_nan)

Kolumny, w których są puste wartości:  ['bathrooms', 'bathrooms_text', 'bedrooms', 'beds']


## Dodanie mediany do bedrooms i beds

In [46]:
# bedrooms i beds wypełniamy poprzez medianę z pozostałych listingów
listings['bedrooms'] = listings['bedrooms'].fillna(listings['bedrooms'].median())
listings['beds'] = listings['beds'].fillna(listings['beds'].median())

In [47]:
# zostały bathrooms i bathrooms_text
columns_with_nan = listings.columns[listings.isnull().any()].tolist()
print('Kolumny, w których są puste wartości: ', columns_with_nan)

Kolumny, w których są puste wartości:  ['bathrooms', 'bathrooms_text']


## Ujednolicenie bathrooms i bathrooms_text

In [48]:
def normalize_half_bath(value):
    if pd.isna(value):
        return value
    value = value.strip().lower()
    if value == 'half-bath':
        return '0.5 bath'
    elif value == 'shared half-bath':
        return '0.5 shared bath'
    elif value == 'private half-bath':
        return '0.5 private bath'
    return value

# pojedyncze wartości odbiegają od typowego 'liczba + suffix', więc naprawiamy to
listings['bathrooms_text'] = listings['bathrooms_text'].apply(normalize_half_bath)

In [49]:
def extract_bathrooms_num(text):
    if pd.isna(text) or text == 'nan':
        return np.nan
    match = re.match(r'^(\d*\.?\d+)', text)
    if match:
        return float(match.group(1))
    return np.nan

def extract_suffix(text):
    if pd.isna(text) or text == 'nan':
        return np.nan
    match = re.match(r'^\d*\.?\d+\s*(.*)$', text)
    if match:
        return match.group(1).strip()
    return np.nan

# dodajemy bathrooms_num i bathrooms_suffix - to będą nasze dane o łazienkach
listings['bathrooms_text_num'] = listings['bathrooms_text'].apply(extract_bathrooms_num)
listings['bathrooms_suffix'] = listings['bathrooms_text'].apply(extract_suffix)
listings['bathrooms_num_old'] = pd.to_numeric(listings['bathrooms'], errors='coerce')
listings['bathrooms_num'] = listings['bathrooms_text_num'].combine_first(listings['bathrooms_num_old'])
listings['bathrooms_suffix'] = listings['bathrooms_suffix'].fillna('missing')

In [50]:
print(listings.head())
print('Możliwe suffixy: ', listings['bathrooms_suffix'].unique())

               id                 neighbourhood_cleansed  \
4019      7352954        l'Antiga Esquerra de l'Eixample   
5125  8,54486E+17                            Sant Antoni   
5385     32537242                      la Vila de Gràcia   
4645  1,18828E+18  Sant Pere, Santa Caterina i la Ribera   
3435  5,57803E+17        l'Antiga Esquerra de l'Eixample   

     neighbourhood_group_cleansed   latitude  longitude  \
4019                     Eixample  41.392190   2.152220   
5125                     Eixample  41.378390   2.155850   
5385                       Gràcia  41.406100   2.151390   
4645                 Ciutat Vella  41.385973   2.178728   
3435                     Eixample  41.388386   2.152384   

                    property_type        room_type  accommodates  bathrooms  \
4019           Entire rental unit  Entire home/apt             6        3.0   
5125  Private room in rental unit     Private room             1        NaN   
5385  Private room in rental unit     Private r

In [51]:
# usuwamy niepotrzebne stare kolumny
listings.drop(columns=['bathrooms', 'bathrooms_text', 'bathrooms_text_num', 'bathrooms_num_old'], inplace=True)

In [52]:
# zamiast pola suffix - chcemy mieć typ - shared, private albo nieznany
def classify_bathroom_type(suffix):
    suffix = suffix.lower()
    if 'shared' in suffix:
        return 'shared'
    elif 'private' in suffix:
        return 'private'
    else:
        return 'unknown'

listings['bathroom_type'] = listings['bathrooms_suffix'].apply(classify_bathroom_type)

In [53]:
# usuwamy kolumnę suffix - mamy liczbę łazienek i ich typ
listings.drop(columns=['bathrooms_suffix'], inplace=True)
print(listings.head())
print('Możliwe wartości type: ', listings['bathroom_type'].unique())

               id                 neighbourhood_cleansed  \
4019      7352954        l'Antiga Esquerra de l'Eixample   
5125  8,54486E+17                            Sant Antoni   
5385     32537242                      la Vila de Gràcia   
4645  1,18828E+18  Sant Pere, Santa Caterina i la Ribera   
3435  5,57803E+17        l'Antiga Esquerra de l'Eixample   

     neighbourhood_group_cleansed   latitude  longitude  \
4019                     Eixample  41.392190   2.152220   
5125                     Eixample  41.378390   2.155850   
5385                       Gràcia  41.406100   2.151390   
4645                 Ciutat Vella  41.385973   2.178728   
3435                     Eixample  41.388386   2.152384   

                    property_type        room_type  accommodates  bedrooms  \
4019           Entire rental unit  Entire home/apt             6       3.0   
5125  Private room in rental unit     Private room             1       1.0   
5385  Private room in rental unit     Private room

In [54]:
# sprawdzamy czy jakieś wiersze mają jeszcze brakujące wartości
rows_with_nan = listings[listings.isnull().any(axis=1)]

if not rows_with_nan.empty:
    print(f"Liczba wierszy z brakami: {len(rows_with_nan)}")
    for index, row in rows_with_nan.iterrows():
        missing_cols = row[row.isnull()].index.tolist()
        print(f"Listing ID: {row['id']}, Brakujące kolumny: {missing_cols}")
else:
    print("Brak brakujących danych (NaN) w listings.")

Liczba wierszy z brakami: 1
Listing ID: 9,39973E+17, Brakujące kolumny: ['bathrooms_num']


In [55]:
# ten wiersz nie miał ani pola bathrooms, ani bathrooms_text - uzupełniamy liczbę medianą tak jak w przypadku beds
listings['bathrooms_num'] = listings['bathrooms_num'].fillna(listings['bathrooms_num'].median())

In [56]:
# teraz już nie ma brakujących danych
rows_with_nan = listings[listings.isnull().any(axis=1)]

if not rows_with_nan.empty:
    print(f"Liczba wierszy z brakami: {len(rows_with_nan)}")
    for index, row in rows_with_nan.iterrows():
        missing_cols = row[row.isnull()].index.tolist()
        print(f"Listing ID: {row['id']}, Brakujące kolumny: {missing_cols}")
else:
    print("Brak brakujących danych (NaN) w listings.")

Brak brakujących danych (NaN) w listings.


In [21]:
# teraz już nie ma brakujących danych
rows_with_nan = listings[listings.isnull().any(axis=1)]

if not rows_with_nan.empty:
    print(f"Liczba wierszy z brakami: {len(rows_with_nan)}")
    for index, row in rows_with_nan.iterrows():
        missing_cols = row[row.isnull()].index.tolist()
        print(f"Listing ID: {row['id']}, Brakujące kolumny: {missing_cols}")
else:
    print("Brak brakujących danych (NaN) w listings.")

Brak brakujących danych (NaN) w listings.


In [22]:
print(listings)

               id                 neighbourhood_cleansed  \
4019      7352954        l'Antiga Esquerra de l'Eixample   
5125  8,54486E+17                            Sant Antoni   
5385     32537242                      la Vila de Gràcia   
4645  1,18828E+18  Sant Pere, Santa Caterina i la Ribera   
3435  5,57803E+17        l'Antiga Esquerra de l'Eixample   
...           ...                                    ...   
3772  1,28402E+18                         el Barri Gòtic   
5191     13086371                          el Fort Pienc   
5226     53802792                 la Dreta de l'Eixample   
5390     39692277                            Sant Antoni   
860   1,15653E+18                               el Raval   

     neighbourhood_group_cleansed   latitude  longitude  \
4019                     Eixample  41.392190   2.152220   
5125                     Eixample  41.378390   2.155850   
5385                       Gràcia  41.406100   2.151390   
4645                 Ciutat Vella  41.38597

## Zamiana instant_bookable

In [57]:
print(listings)

               id                 neighbourhood_cleansed  \
4019      7352954        l'Antiga Esquerra de l'Eixample   
5125  8,54486E+17                            Sant Antoni   
5385     32537242                      la Vila de Gràcia   
4645  1,18828E+18  Sant Pere, Santa Caterina i la Ribera   
3435  5,57803E+17        l'Antiga Esquerra de l'Eixample   
...           ...                                    ...   
3772  1,28402E+18                         el Barri Gòtic   
5191     13086371                          el Fort Pienc   
5226     53802792                 la Dreta de l'Eixample   
5390     39692277                            Sant Antoni   
860   1,15653E+18                               el Raval   

     neighbourhood_group_cleansed   latitude  longitude  \
4019                     Eixample  41.392190   2.152220   
5125                     Eixample  41.378390   2.155850   
5385                       Gràcia  41.406100   2.151390   
4645                 Ciutat Vella  41.38597

## Zamiana instant_bookable

In [58]:
# zamiast instant bookable t/f chcemy 1/0
listings['instant_bookable'] = listings['instant_bookable'].map({'t': 1, 'f': 0})

In [59]:
print(listings)

               id                 neighbourhood_cleansed  \
4019      7352954        l'Antiga Esquerra de l'Eixample   
5125  8,54486E+17                            Sant Antoni   
5385     32537242                      la Vila de Gràcia   
4645  1,18828E+18  Sant Pere, Santa Caterina i la Ribera   
3435  5,57803E+17        l'Antiga Esquerra de l'Eixample   
...           ...                                    ...   
3772  1,28402E+18                         el Barri Gòtic   
5191     13086371                          el Fort Pienc   
5226     53802792                 la Dreta de l'Eixample   
5390     39692277                            Sant Antoni   
860   1,15653E+18                               el Raval   

     neighbourhood_group_cleansed   latitude  longitude  \
4019                     Eixample  41.392190   2.152220   
5125                     Eixample  41.378390   2.155850   
5385                       Gràcia  41.406100   2.151390   
4645                 Ciutat Vella  41.38597

# One-hot encoding

In [60]:
# listę udogodnień chcemy załatwić przez one-hot-encoding - wybieramy 50 najpopularniejszych
all_amenities = Counter(item
    for amenities in listings['amenities']
    for item in ast.literal_eval(amenities)
)

In [61]:
print(all_amenities.most_common(50))

[('Wifi', 4860), ('Kitchen', 4717), ('Hot water', 3893), ('Hair dryer', 3845), ('Hangers', 3776), ('Essentials', 3760), ('Iron', 3708), ('Dishes and silverware', 3543), ('Refrigerator', 3404), ('Bed linens', 3399), ('TV', 3276), ('Cooking basics', 3243), ('Heating', 3200), ('Washer', 3184), ('Microwave', 3163), ('Air conditioning', 2957), ('Elevator', 2758), ('Shampoo', 2420), ('Dedicated workspace', 2416), ('Oven', 2388), ('Coffee maker', 2289), ('Hot water kettle', 2076), ('Long term stays allowed', 2050), ('Freezer', 2020), ('Toaster', 1902), ('Dishwasher', 1886), ('Dining table', 1810), ('Stove', 1696), ('Cleaning products', 1625), ('Drying rack for clothing', 1621), ('Wine glasses', 1598), ('Host greets you', 1587), ('Room-darkening shades', 1538), ('Shower gel', 1467), ('Extra pillows and blankets', 1465), ('Body soap', 1361), ('Smoke alarm', 1357), ('Fire extinguisher', 1132), ('First aid kit', 1082), ('Luggage dropoff allowed', 1079), ('Self check-in', 1066), ('Patio or balcony

In [62]:
# zamieniamy listę na wartości 1/0 w odpowiednich utworzonych przez one-hot kolumnach
top_amenities = [item for item, count in all_amenities.most_common(50)]

def has_amenity(amenities_str, amenity):
    amenities_list = ast.literal_eval(amenities_str)
    return int(amenity in amenities_list)

for amenity in top_amenities:
    listings[f'amenity_{amenity}'] = listings['amenities'].apply(lambda x: has_amenity(x, amenity))

listings.drop(columns=['amenities'], inplace=True)

In [63]:
print(listings.columns)

Index(['id', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed',
       'latitude', 'longitude', 'property_type', 'room_type', 'accommodates',
       'bedrooms', 'beds', 'instant_bookable', 'price', 'bathrooms_num',
       'bathroom_type', 'amenity_Wifi', 'amenity_Kitchen', 'amenity_Hot water',
       'amenity_Hair dryer', 'amenity_Hangers', 'amenity_Essentials',
       'amenity_Iron', 'amenity_Dishes and silverware', 'amenity_Refrigerator',
       'amenity_Bed linens', 'amenity_TV', 'amenity_Cooking basics',
       'amenity_Heating', 'amenity_Washer', 'amenity_Microwave',
       'amenity_Air conditioning', 'amenity_Elevator', 'amenity_Shampoo',
       'amenity_Dedicated workspace', 'amenity_Oven', 'amenity_Coffee maker',
       'amenity_Hot water kettle', 'amenity_Long term stays allowed',
       'amenity_Freezer', 'amenity_Toaster', 'amenity_Dishwasher',
       'amenity_Dining table', 'amenity_Stove', 'amenity_Cleaning products',
       'amenity_Drying rack for clothing', 'amen

In [64]:
# w tych kolumnach stosujemy one-hot
categorical_cols = ['property_type', 'room_type', 'neighbourhood_cleansed', 'neighbourhood_group_cleansed', 'bathroom_type']

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoded_array = encoder.fit_transform(listings[categorical_cols])
encoded_cols = encoder.get_feature_names_out(categorical_cols)
encoded_df = pd.DataFrame(encoded_array, columns=encoded_cols, index=listings.index)
listings = pd.concat([listings.drop(columns=categorical_cols), encoded_df], axis=1)

# Zamiana kolejności

In [65]:
cols = [col for col in listings.columns if col != 'price'] + ['price']
listings = listings[cols]

# Dodane nowe atrybuty

In [68]:
# === Funkcja odległości do centrum Barcelony
def haversine(lat1, lon1, lat2=41.3870, lon2=2.1701):
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lon2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    return R * 2 * atan2(sqrt(a), sqrt(1 - a))

In [69]:
df = listings
df = df[(df["price"] >= df["price"].quantile(0.01)) & (df["price"] <= df["price"].quantile(0.99))].copy()

# === Feature engineering
df["bathrooms_per_guest"] = df["bathrooms_num"] / df["accommodates"].replace(0, np.nan)
df["bedrooms_per_guest"] = df["bedrooms"] / df["accommodates"].replace(0, np.nan)
df["beds_per_guest"] = df["beds"] / df["accommodates"].replace(0, np.nan)
df["beds_per_bedroom"] = df["beds"] / df["bedrooms"].replace(0, np.nan)
df["guests_per_bedroom"] = df["accommodates"] / df["bedrooms"].replace(0, np.nan)
df["dist_to_center"] = df.apply(lambda row: haversine(row["latitude"], row["longitude"]), axis=1)

# === Klasteryzacja lokalizacji
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=15, random_state=42, n_init='auto')
df["location_cluster"] = kmeans.fit_predict(df[["latitude", "longitude"]])
joblib.dump(kmeans, "final_models/kmeans_location_cluster.pkl")

# === Dodatkowe cechy
amenity_cols = [col for col in df.columns if col.startswith("amenity_")]
df["num_amenities"] = df[amenity_cols].sum(axis=1)
df["has_tv_or_wifi"] = df[["amenity_TV", "amenity_Wifi"]].max(axis=1)
df["is_suited_for_longterm"] = df[["amenity_Washer", "amenity_Kitchen", "amenity_Dishes and silverware"]].sum(axis=1) >= 2
df["lat_scaled"] = (df["latitude"] - df["latitude"].mean()) / df["latitude"].std()
df["lon_scaled"] = (df["longitude"] - df["longitude"].mean()) / df["longitude"].std()
df["is_group_friendly"] = (df["accommodates"] >= 4).astype(int)

In [70]:
# w tych kolumnach dane są niepełne
columns_with_nan = df.columns[df.isnull().any()].tolist()
print('Kolumny, w których są puste wartości: ', columns_with_nan)

Kolumny, w których są puste wartości:  ['beds_per_bedroom', 'guests_per_bedroom']


In [71]:
df.fillna(0, inplace=True)

In [72]:
# sprawdzamy czy jakieś wiersze mają jeszcze brakujące wartości
rows_with_nan = df[df.isnull().any(axis=1)]

if not rows_with_nan.empty:
    print(f"Liczba wierszy z brakami: {len(rows_with_nan)}")
    for index, row in rows_with_nan.iterrows():
        missing_cols = row[row.isnull()].index.tolist()
        print(f"Listing ID: {row['id']}, Brakujące kolumny: {missing_cols}")
else:
    print("Brak brakujących danych (NaN) w df.")

Brak brakujących danych (NaN) w df.


In [73]:
for col in ["bathrooms_per_guest", "bedrooms_per_guest", "beds_per_guest", "beds_per_bedroom", "guests_per_bedroom"]:
    df[col] = df[col].replace([np.inf, -np.inf], np.nan)
    q_low, q_high = df[col].quantile([0.01, 0.99])
    df[col] = df[col].clip(q_low, q_high).fillna(df[col].median())